In [1]:
import pandas as pd
df = pd.read_csv('ai_ml_papers.csv')

/var/folders/kv/h86sly590pv79w7ckm213kg80000gq/T/ipykernel_83059/2969752437.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('ai_ml_papers.csv')


In [10]:
from extract_pdf import extract_pdf_text

In [35]:
print(df.iloc[7]['abstract'])

  Ordinal regression is an important type of learning, which has properties of
both classification and regression. Here we describe a simple and effective
approach to adapt a traditional neural network to learn ordinal categories. Our
approach is a generalization of the perceptron method for ordinal regression.
On several benchmark datasets, our method (NNRank) outperforms a neural network
classification method. Compared with the ordinal regression methods using
Gaussian processes and support vector machines, NNRank achieves comparable
performance. Moreover, NNRank has the advantages of traditional neural
networks: learning in both online and batch modes, handling very large training
datasets, and making rapid predictions. These features make NNRank a useful and
complementary tool for large-scale data processing tasks such as information
retrieval, web page ranking, collaborative filtering, and protein ranking in
Bioinformatics.



In [11]:
text = extract_pdf_text(df.iloc[7]['id'])
#print(text)

In [13]:
!pip3 install spacy
!python3 -m spacy download en_core_web_sm
!pip3 install --user -U nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 5.8 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 895.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.0/133.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 839.3/839.3 kB 8.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.9/635.9 kB 4.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.7/431.7 kB 6.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.0/183.0 kB 4.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 9.8 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 9.1 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/

In [ ]:
#run this if not certified: /Applications/Python\ 3.11/Install\ Certificates.command
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1002)>
[nltk_data] Error loading omw-1.4: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1002)>


False

In [3]:
import spacy
from nltk.corpus import wordnet

nlp = spacy.load("en_core_web_sm")

def is_meaningful_title(title):
    doc = nlp(title)

    #must contain at least one valid word
    valid_words = [token.text for token in doc if token.is_alpha and wordnet.synsets(token.text)]
    if not valid_words:
        return False

    #must contain a noun or a verb (common in section titles)
    if not any(token.pos_ in {"PROPN", "NOUN", "VERB"} for token in doc):
        return False

    #should not be just a function word
    if all(token.pos_ in {"DET", "ADP", "CCONJ"} for token in doc):
        return False

    #ensure numbering follows a common pattern
    if doc[0].is_digit and len(doc) > 1 and doc[1].pos_ in {"DET", "ADV"}:
        return False

    return True

In [4]:
import re

def extract_sections(text):
    """Extracts section titles and their positions from text."""
    section_patterns = [
        r'^(\d+\.?\s+[A-Z][A-Za-z ]+)',   # 1. Title / 1 Title
        r'^(\d+\.\d+\.?\s+[A-Z][A-Za-z ]+)', # 1.1 Title
        r'^(I{1,3}|IV|V{1,3}|VI{1,3})\.\s+[A-Z][A-Za-z ]+',  # III. Title
        r'^[A-Z]\.\s+[A-Z][A-Za-z ]+',  # A. Title
        r'^[A-Z][A-Z ]+$'  # INTRODUCTION
    ]

    keyword_sections = {"INTRODUCTION", "METHOD", "EXPERIMENT", "RESULTS", "DISCUSSION", "CONCLUSION", "REFERENCES", "ACKNOWLEDGEMENTS", "BIBLIOGRAPHY"}

    numbering_patterns = [
        r'^\d+(\.\d+)*\.?\s',  # 1, 1.1, 1.2.3
        r'^(I{1,3}|IV|V{1,3}|VI{1,3})\.\s',  # I. II. III.
        r'^[A-Z]\.\s'  # A. B. C.
    ]

    sections = []
    lines = text.split('\n')

    for i, line in enumerate(lines):
        line = line.strip()
        if any(re.match(pattern, line) for pattern in section_patterns) and len(line.split(' ')) < 10 or line.upper() in keyword_sections:
            if ('references' in line.lower() or 'bibliography' in line.lower() or 'acknowledgement' in line.lower()):
                break
            sections.append((i, line))

    #if a section name doesn't make sense, remove it
    sections = [sec for sec in sections if is_meaningful_title(sec[1])]

    #if there is an existing numbering pattern, remove other titles as noise
    numbering_pattern_count = 0
    for section in sections:
        for pattern in numbering_patterns:
            if re.match(pattern, section[1]):
                numbering_pattern_count += 1
    if numbering_pattern_count > 5:
        for section in sections[:]:
            matched = False

            for pattern in numbering_patterns:
                if re.match(pattern, section[1]):
                    matched = True
            if not matched:
                sections.remove(section)

    return sections


In [5]:
sections = extract_sections(text)
for section in sections:
    print(section)

(25, '1. Introduction')
(121, '2. Method')
(122, '2.1. Formulation')
(167, '2.2. Learning')
(266, '2.3. Prediction')
(273, '3. Experiments and Results')
(274, '3.1. Benchmark Data and Evaluation Metric')
(313, '3.2. Comparison with Neural Network')
(335, '3.3. Comparison with Gaussian Processes and')
(353, '4. Discussion and Future Work')


In [6]:
def detect_numbering(title):
    if match := re.match(r'^(I{1,3}|IV|V{1,3}|VI{1,3})\.\s', title):  #roman numerals
        return ("roman", match.group(0).strip())
    elif match := re.match(r'^\d+(\.\d+)*\.?\s', title):  #numeric (1, 1.1, etc.)
        value = match.group(0).strip()
        return ("numeric" + str(value.rstrip('.').count('.')), value)
    elif match := re.match(r'^[A-Z]\.\s', title):  #alphabetic (A., B., etc.)
        return ("alpha", match.group(0).strip())
    return (None, None)  #no numbering

In [7]:
#assume all section names have numbering
def construct_hierarchy_with_numbering(sections):
    prev_type = None

    numbering_to_prefix = {}

    for _, title in sections:
        type, value = detect_numbering(title)
        if prev_type is None:
            print(title)
            numbering_to_prefix[type] = ''
            prev_type = type
            continue
        if type not in numbering_to_prefix:
            if numbering_to_prefix[prev_type] == '':
                numbering_to_prefix[type] = '  └─── '
            else:
                numbering_to_prefix[type] = '  ' + numbering_to_prefix[prev_type]
        print(numbering_to_prefix[type] + title)
        prev_type = type

In [14]:
construct_hierarchy_with_numbering(sections)

1. Introduction
2. Method
  └─── 2.1. Formulation
  └─── 2.2. Learning
  └─── 2.3. Prediction
3. Experiments and Results
  └─── 3.1. Benchmark Data and Evaluation Metric
  └─── 3.2. Comparison with Neural Network
  └─── 3.3. Comparison with Gaussian Processes and
4. Discussion and Future Work


In [19]:
!python3 -m spacy download en_core_web_lg

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 1.3 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [8]:
import re
import spacy

# Load spaCy English model
nlp = spacy.load("en_core_web_lg")

def is_valid_word(word):
    doc = nlp(word)
    lemma = doc[0].lemma_
    return word in nlp.vocab or lemma in nlp.vocab

def fix_hyphenation(text):
    #find all hyphenated words using regex
    matches = list(re.finditer(r'(\w+)-(\w+)', text))
    
    #process matches in reverse order (to avoid index shifting issues)
    for match in reversed(matches):
        word1, word2 = match.groups()
        joined_word = word1 + word2
        
        if is_valid_word(joined_word):
            #replace only if the joined word is valid
            start, end = match.span()
            text = text[:start] + joined_word + text[end:]
        else:
            print(joined_word)

    return text


In [9]:
#adapted from Eka's code to cover some corner cases
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def preprocess(text):
    stop_words = set(stopwords.words('english'))

    #lowercase and remove URLs/special characters
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    #text = re.sub(r'[^a-zA-Z\s]', '', text).strip()
    
    #Removing Extra Spaces
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'-\s', '-', text)
    
    #Joining wordssplit by -
    text = fix_hyphenation(text)
    
    #Remove citations and references
    text = re.sub(r'\[\d+\]', '', text)  # Removes [12]
    text = re.sub(r'\([A-Z][^)]*,\s\d{4}[a-z]?\)', '', text)  # Removes (Smith et al., 2020)
    
    text = re.sub(r'[^\w\s.,!?-]', '', text)

    # Tokenize and lemmatize
    #tokens = word_tokenize(text)
    #tokens = [word.lower() if not word.isupper() else word for word in tokens]
    #tokens = [word for word in tokens if word not in stop_words]
    
    return text


In [11]:
def get_section_texts(text_lines, section_list):
    section_texts = {}
    
    for i in range(len(section_list) - 1):
        start_line = section_list[i][0] + 1
        section_name = section_list[i][1]
        end_line = section_list[i + 1][0] - 1

        if start_line <= end_line:
            section_texts[section_name] = " ".join(text_lines[start_line:end_line + 1])
        else:
            section_texts[section_name] = ""

    last_section_name = section_list[-1][1]
    last_section_start = section_list[-1][0]
    last_section_text = " ".join(text_lines[last_section_start:])
    last_section_text = last_section_text.split("\nReferences\n")[0]
    last_section_text = last_section_text.split("\nBibliography\n")[0]
    section_texts[last_section_name] = last_section_text

    return section_texts


In [12]:
section_texts = get_section_texts(text.split('\n'), sections)

In [33]:
full_section_texts = ''
for k in section_texts:
    full_section_texts += k
    full_section_texts += ': '
    full_section_texts += section_texts[k]
    full_section_texts += '\n'
print(full_section_texts)


1. Introduction: Ordinal regression (or ranking learning) is an important supervised problem of learning a ranking or ordering on instances, which has the property of both classification and metric regression. The learning task of ordinal regression is to assign data points into a set of finite ordered categories. For example, a teacher rates students’ performance using A, B, C, D, and E (A > B > C > D > E) (Chu & Ghahramani, 2005a). Ordinal regression is different from classification due to the order of categories. In contrast to metric regression, the response variables (categories) in ordinal regression is discrete and finite. The research of ordinal regression dated back to the ordinal statistics methods in 1980s (McCullagh, 1980; McCullagh & Nelder, 1983) and machine learning research in 1990s (Caruana et al., 1996; Herbrich et al., 1998; Cohen et al., 1999). It has attracted the considerable attention in recent years due to its potential applications in many data-intensive domain

In [26]:
!pip3 install sentencepiece

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 9.2 MB/s eta 0:00:00:00:0100:01

[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python3.12 -m pip install --upgrade pip


In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

#load Pegasus model & tokenizer
model_name = "google/pegasus-xsum"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

/Users/ruoqwang/Library/Python/3.12/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/ruoqwang/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [14]:
def chunk_text(text, max_tokens=512):
    sentences = nltk.sent_tokenize(text)
    chunks = []
    current_chunk = []
    current_length = 0

    for sentence in sentences:
        tokenized_sentence = tokenizer.tokenize(sentence)
        sentence_length = len(tokenized_sentence)
        length = current_length + sentence_length
        #if adding the sentence exceeds max_tokens, start a new chunk
        if current_length + sentence_length > max_tokens:
            chunks.append(" ".join(current_chunk))
            current_chunk = [sentence]
            current_length = sentence_length
        else:
            current_chunk.append(sentence)
            current_length += sentence_length

    #append the last chunk if not empty
    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

In [15]:
def summarize_chunk(text, max_length=50):
    inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True)
    print('try to generate with model')
    summary_ids = model.generate(
        inputs["input_ids"],
        max_length=max_length,  
        min_length=10,  
        num_beams=8,  
        length_penalty=0.5,  
        early_stopping=True
    )
    res = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return res

In [16]:
#recursively summarize full text of each section
def summarize_full_text(text):
    chunks = chunk_text(text)
    summaries = [summarize_chunk(chunk) for chunk in chunks]
    combined_summary = " ".join(summaries)
    final_summary = summarize_chunk(combined_summary, max_length=30)

    return final_summary

In [21]:

summary = summarize_full_text(section_texts['1. Introduction'])
print(summary)

try to generate with model
try to generate with model
try to generate with model
try to generate with model
try to generate with model
We present a new method for the classification of large-scale unstructured data.


In [125]:
print(sections)

[(41, '1. Introduction'), (165, '2. Method'), (166, '2.1. Formulation'), (217, '2.2. Learning'), (321, '2.3. Prediction'), (329, '3. Experiments and Results'), (330, '3.1. Benchmark Data and Evaluation Metric'), (378, '3.2. Comparison with Neural Network'), (403, '3.3. Comparison with Gaussian Processes and'), (425, '4. Discussion and Future Work')]


In [42]:
print(section_texts)

{'1. Introduction': 'Ordinal regression (or ranking learning) is an impor- tant supervised problem of learning a ranking or or- dering on instances, which has the property of both classiﬁcation and metric regression. The learning task of ordinal regression is to assign data points into a set of ﬁnite ordered categories. For example, a teacher rates students’ performance using A, B, C, D, and E (A > B > C > D > E) (Chu & Ghahramani, 2005a). Ordinal regression is diﬀerent from classiﬁcation due to the order of categories. In contrast to metric re- gression, the response variables (categories) in ordinal regression is discrete and ﬁnite. The research of ordinal regression dated back to the ordinal statistics methods in 1980s (McCullagh, 1980; McCullagh & Nelder, 1983) and machine learning re- search in 1990s (Caruana et al., 1996; Herbrich et al., 1998; Cohen et al., 1999). It has attracted the con- siderable attention in recent years due to its poten- tial applications in many data-inten

In [43]:
#assume all section names have numbering
def construct_markup_with_numbering(sections):
    prev_type = ''
    ret = ''

    numbering_to_prefix = {}

    for _, title in sections:
        type, value = detect_numbering(title)
        if prev_type == '':
            numbering_to_prefix[type] = '#'
        elif type not in numbering_to_prefix:
            numbering_to_prefix[type] = '#' + numbering_to_prefix[prev_type]
        
        #construct summary
        summary = summarize_full_text(section_texts[title])

        ret += numbering_to_prefix[type] + ' ' + title + summary + '\n'
        prev_type = type
    return ret

In [44]:
markup = construct_markup_with_numbering(sections)
print(markup)

try to generate with model
try to generate with model
try to generate with model
try to generate with model
try to generate with model
try to generate with model
try to generate with model
try to generate with model
try to generate with model
try to generate with model
try to generate with model
try to generate with model
try to generate with model
try to generate with model
try to generate with model
try to generate with model
try to generate with model
try to generate with model
try to generate with model
try to generate with model
try to generate with model
try to generate with model
try to generate with model
try to generate with model
try to generate with model
try to generate with model
try to generate with model
try to generate with model
try to generate with model
# 1. IntroductionOrdinal regression problem is one of the most challenging problems in machine learning.
# 2. MethodA selection of photos from around the world this week:
## 2.1. FormulationWe have developed a new met

In [46]:
generate_markmap(markup)

In [6]:
import subprocess

MARKMAP_PATH = "/Users/evelinewang/.nvm/versions/node/v22.14.0/bin/markmap"

def generate_markmap(markdown_string, output_file="markmap.html"):
    with open("temp.md", "w", encoding="utf-8") as f:
        f.write(markdown_string)

In [ ]:
generate_markmap(markdown_string)
#I didn't manage to make this work so I ran 'markmap temp.md -o markmap.html' on my terminal to generate directly

(node:68875) ExperimentalWarning: The ESM module loader is experimental.
internal/modules/run_main.js:54
    internalBinding('errors').triggerUncaughtException(
                              ^

Error [ERR_MODULE_NOT_FOUND]: Cannot find package 'fs' imported from /Users/evelinewang/.nvm/versions/node/v22.14.0/lib/node_modules/markmap-cli/dist/cli.js
    at packageResolve (internal/modules/esm/resolve.js:594:9)
    at moduleResolve (internal/modules/esm/resolve.js:633:14)
    at Loader.defaultResolve [as _resolve] (internal/modules/esm/resolve.js:726:11)
    at Loader.resolve (internal/modules/esm/loader.js:97:40)
    at Loader.getModuleJob (internal/modules/esm/loader.js:243:28)
    at ModuleWrap.<anonymous> (internal/modules/esm/module_job.js:47:40)
    at link (internal/modules/esm/module_job.js:46:36) {
  code: 'ERR_MODULE_NOT_FOUND'
}


CalledProcessError: Command '['/Users/evelinewang/.nvm/versions/node/v22.14.0/bin/markmap', 'temp.md', '-o', 'markmap.html']' returned non-zero exit status 1.

In [28]:
!pip3 install rouge_score

Defaulting to user installation because normal site-packages is not writeable
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 2.3 MB/s eta 0:00:0000:01
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24986 sha256=5602dff33df9802b737573ed816dddc27217ba27079b698afa92a890dae606db
  Stored in directory: /Users/ruoqwang/Library/Caches/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score

[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python3.12 -m pip install --upgrade pip


In [26]:
from rouge import Rouge

from evaluate import load
import bert_score

In [ ]:
#copied from Eka's notebook
# Initialize metrics
rouge = load("rouge")
bleu = load("bleu")
# Sample input
abstract = """Ordinal regression is an important type of learning, which has properties of
both classification and regression. Here we describe a simple and effective
approach to adapt a traditional neural network to learn ordinal categories. Our
approach is a generalization of the perceptron method for ordinal regression.
On several benchmark datasets, our method (NNRank) outperforms a neural network
classification method. Compared with the ordinal regression methods using
Gaussian processes and support vector machines, NNRank achieves comparable
performance. Moreover, NNRank has the advantages of traditional neural
networks: learning in both online and batch modes, handling very large training
datasets, and making rapid predictions. These features make NNRank a useful and
complementary tool for large-scale data processing tasks such as information
retrieval, web page ranking, collaborative filtering, and protein ranking in
Bioinformatics."""

generated_summary = """1. Introduction: Ordinal regression problem is one of the most challenging problems in machine learning.
2. Method: A selection of photos from around the world this week:
2.1. Formulation: We have developed a new method for ordinal re- gression, which can be used to train classification systems.
2.2. Learning: The cost function for a data point x can be relative entropy or square error between the tar- get vector and the output vector.
2.3. Prediction: The category of a data point is an important parameter of its classification.
3. Experiments and Results: A selection of photos from around the world this week:
3.1. Benchmark Data and Evaluation Metric: A neural network approach to ordinal regression is presented.
3.2. Comparison with Neural Network: A new ordinal regression neural network method is presented in this paper.
3.3. Comparison with Gaussian Processes and: We present a new ordinal regression method that achieves the best results on eight datasets.
4. Discussion and Future Work: In our work, we have proposed two directions for the design of neural networks for ordinal regression.
"""

def evaluate_automatic_metrics(abstract, generated_summary):
    results = {}

    # ROUGE
    rouge_scores = rouge.compute(predictions=[generated_summary], references=[abstract])
    results["ROUGE-1"] = round(rouge_scores["rouge1"], 3)
    results["ROUGE-2"] = round(rouge_scores["rouge2"], 3)
    results["ROUGE-L"] = round(rouge_scores["rougeL"], 3)

    # BLEU
    bleu_score = bleu.compute(predictions=[generated_summary], references=[[abstract]])
    results["BLEU"] = round(bleu_score["bleu"], 3)

    # BERTScore
    P, R, F1 = bert_score.score([generated_summary], [abstract], lang="en", verbose=False)
    results["BERTScore-F1"] = round(F1[0].item(), 3)

    return results

auto_scores = evaluate_automatic_metrics(abstract, generated_summary)
for metric, score in auto_scores.items():
    print(f"{metric}: {score}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ROUGE-1: 0.39
ROUGE-2: 0.103
ROUGE-L: 0.211
BLEU: 0.038
BERTScore-F1: 0.833
